In [4]:
import numpy as np
import csv
import os
import math
import random
import copy

Q = 7 # Number of available queues
F = 64 # Number of flows
M = 6 # Number of malicious flows
T = 10 # Cycle length [s]
T_END = 300 # Simulation time [s]
TH = 80 # Threshold for queue size increase [Mb]

L = 50.0 # Link speed [Mbps]
R_A = 2.0 # Rate avg [Mbps]
R_S = 0.5  # Rate stdev [Mbps]
D = 100.0  # DDos rate [Mbps]

# make output file
#dirPath = f'test{MinLink}{MaxLink}_{N}_{F}_{T}'
#os.mkdir(dirPath)
#path = f'test{MinLink}{MaxLink}_{N}_{F}_{T}/test{MinLink}{MaxLink}_{N}_{F}_{T}.cnf.txt'

path1 = f'queueSize_{Q}_{F}.txt'
with open(path1, mode='w') as outfile:
    outfile.write('')

path2 = f'queueMap_{Q}_{F}.txt'
with open(path2, mode='w') as outfile:
    outfile.write('')

path3 = f'numAllocatedFlow_{Q}_{F}.txt'
with open(path3, mode='w') as outfile:
    outfile.write('')

In [5]:
### Iitialization

In [6]:
qSize = np.zeros(Q, dtype=int)
qMem = copy.copy(qSize)
qFlag = np.zeros(Q, dtype=int)
fFlag = np.zeros(F, dtype=int)

# Random allocation
qMap = np.zeros(F, dtype=int)
for i in range(F):
    qMap[i] = random.randint(1, Q-1)
print(qMap)

[1 2 1 1 4 6 5 3 1 6 1 4 5 2 3 3 2 1 3 6 6 2 4 2 5 6 2 2 6 4 4 1 2 4 4 3 1
 1 6 6 2 6 6 6 1 2 5 6 5 4 2 3 2 6 5 2 6 5 3 6 1 6 4 3]


In [7]:
### Step 1

In [8]:
for t in range(T_END):
    if t % T == T - 1:
        # Check queue size
        for q in range(Q):
            if qSize[q] - qMem[q] > TH:
                if q > 0:
                    qFlag[q] = 1
                else:
                    qFlag[q] = 0
            else:
                qFlag[q] = 0
        qMem = copy.copy(qSize)
        
        # Count flow per queue
        numAllocatedFlow = np.zeros(Q, dtype=int)
        for f in range(F):
            if qMap[f] != -1:
                numAllocatedFlow[qMap[f]] += 1
        print(numAllocatedFlow)
        with open(path3, mode='a') as outfile:
            for q in range(Q):
                outfile.write(str(numAllocatedFlow[q]))
                if q == Q-1:
                    outfile.write('\n')
                else:
                    outfile.write('\t')
        
        # Flow identification
        for f in range(F):
            if qMap[f] == -1:
                fFlag[f] = 2
            elif qFlag[qMap[f]] == 1:
                if numAllocatedFlow[qMap[f]] == 1:
                    fFlag[f] = 2 # DDoS identified
                else:
                    fFlag[f] = 1
            else:
                fFlag[f] = 0
                
        # Re-allocation
        for f in range(F):
            if fFlag[f] == 2: # Identified DDoS
                qMap[f] = -1 # Drop
            elif fFlag[f] == 1: # ?
                qMap[f] = random.randint(1, Q-1) # Random allocation to Queue 1:Q-1
            else: # Low-rate
                qMap[f] = 0 # Queue 0
        print(qMap)
        
        # write queueMap file
        with open(path2, mode='a') as outfile:
            for f in range(F):
                outfile.write(str(qMap[f]))
                if f == F-1:
                    outfile.write('\n')
                else:
                    outfile.write('\t')            
    
    # enque
    for f in range(F):
        if qMap[f] >= 0:
            if f < M:
                qSize[qMap[f]] += D # DDoS
            else:
                qSize[qMap[f]] += np.random.normal(R_A, R_S)
        else:
            # drop
            qSize[qMap[f]] += 0
    
    # deque
    available = L
    ct = 0
    while available > 0:
        if qSize[ct] > 0:
            qSize[ct] -= 1
            available -= 1
        ct = (ct + 1) % Q
    print(qSize)
    
    # write queueSize file
    with open(path1, mode='a') as outfile:
        outfile.write(str(t+1))
        for q in range(Q):
            outfile.write('\t' + str(qSize[q]))
            if q == Q-1:
                outfile.write('\n')


[  0 303 109   2 104   0 119]
[  0 607 220   7 211   1 234]
[  0 910 333   9 316   3 350]
[   0 1213  442   12  420    3  464]
[   0 1517  555   16  524    6  576]
[   0 1822  665   21  628    7  687]
[   0 2126  773   24  734    7  801]
[   0 2427  883   27  839   12  916]
[   0 2729  992   29  942   14 1028]
[ 0 11 13  8  9  7 16]
[1 4 2 2 3 6 0 0 2 4 6 5 0 2 0 0 2 5 0 5 1 4 1 1 0 5 3 3 5 6 6 4 5 2 4 0 2
 4 3 5 5 3 6 1 2 2 0 1 0 6 6 0 2 3 0 3 6 0 0 5 4 5 6 0]
[  14 2829 1198  134 1045   22 1133]
[  32 2928 1402  235 1148   28 1237]
[  46 3028 1608  338 1249   38 1343]
[  63 3127 1812  440 1352   46 1448]
[  78 3225 2014  545 1455   55 1552]
[  89 3324 2219  648 1557   60 1658]
[ 106 3424 2423  749 1661   65 1764]
[ 121 3523 2629  854 1763   73 1867]
[ 135 3622 2834  955 1864   83 1973]
[ 151 3723 3042 1057 1965   87 2079]
[15  6 10  7  7 10  9]
[2 3 1 4 3 4 0 0 2 3 3 0 0 3 0 0 4 0 0 0 6 3 3 6 0 0 6 6 0 4 4 6 0 3 4 0 2
 1 1 0 0 4 4 2 6 4 0 4 0 4 4 0 6 6 0 2 5 0 0 0 2 0 2 0]
[ 176 3818

[13966  6611  6863  8924  5932 10920  6007]
[14040  6604  6856  8917  5925 10913  6000]
[58  0  0  0  0  0  0]
[-1 -1 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[14119  6597  6849  8910  5918 10906  5993]
[14196  6590  6842  8903  5911 10899  5986]
[14273  6583  6835  8896  5904 10892  5979]
[14349  6576  6828  8889  5897 10885  5972]
[14427  6569  6821  8882  5890 10878  5965]
[14510  6562  6814  8875  5883 10871  5958]
[14590  6555  6807  8868  5876 10864  5951]
[14665  6548  6800  8861  5869 10857  5944]
[14746  6541  6793  8854  5862 10850  5937]
[14817  6534  6786  8847  5855 10843  5930]
[58  0  0  0  0  0  0]
[-1 -1 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[14899  6527  6779  8840  5848 108

In [6]:
### Step 2

In [7]:



#with open(path1, mode='a') as outfile:
#    for i in range(N):
#        outfile.write(str(xPos[i]) + '\t' + str(finalSize[i]) + '\t' + str(math.sqrt(finalSize[i]**2 + H**2)) + '\n')